In [68]:
import numpy as np
import os
import scipy.stats as stats
import scipy.io as io
import random
import itertools
import glob
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from xgboost import XGBClassifier
import random
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import pandas as pd

In [69]:
def file_create(dat,df,main_file,folder):
    IDs = df[df['Nextstrain_clade'] == dat]['strain']
    ID_list = IDs.tolist()
    print(len(ID_list))
    g = open(folder+'/'+dat+'.fasta','w')
    flag = 0
    for ID in ID_list:
        flag = 0
        f = open(main_file, 'r')
        for i, line in enumerate(f):
                if line.startswith('>') and flag == 0:
                    check_ID = line.split('>')[1]
                    check_ID = check_ID.strip()
                    if check_ID == ID:
                        flag = 1
                elif line.startswith('>') and flag == 1:
                    break
                if flag == 1:
                    g.write(line)
        f.close()
    g.close()

In [ ]:
folder = 'Clade_Analysis'
#fname = glob.glob(folder+'/ncov_global.tsv')
#print(fname)
f_path = folder + "/ncov_global.tsv"
print(f_path)
df = pd.read_csv(f_path, sep="\t")
main_file = 'ncov_global.fasta'
#status = ['20A','19A','20B','20C','19B']
status = ['20C', '19A']
for stat in status:
    print(stat)
    #downloaded sequence fasta file from GISAID for the sequences listed in nextstrain*.csv
    file_create(stat, df, main_file, folder)

In [43]:
from collections import defaultdict
def getSeqs(filename):
    seq_list = defaultdict(list)
    mapseq_list = defaultdict(list)
    mapping = {'A': 'a', 'T': 't', 'C': 'c', 'G': 'g','a': 'a', 't': 't', 'c': 'c', 'g': 'g'}
    with open(filename) as f:
        j = -1
        for i, line in enumerate(f):
            if line.startswith('>'):
                j += 1
            else:
                this_line = list(line)
                this_line = list(filter(lambda ch: ch in 'acgtACGT', this_line))
                seq_list[j].extend(this_line)
                mapseq_list[j] = ''.join(list(map(lambda ch: mapping[ch], seq_list[j])))
    return mapseq_list

In [45]:
status = [['20C'], ['19A']] #
V = []
for stat in status:
    filenames = []
    for s in stat:
        filenames.append('Clade_Analysis/'+s+'.fasta')
    sequences = []
    for filename in filenames:
        print(filename)
        sequences.append(getSeqs(filename))
    V.append(sequences)

Clade_Analysis/20C.fasta
Clade_Analysis/19A.fasta


In [46]:
import pickle
with open('Clade_Analysis/V_clades.pkl','wb') as f:  
    pickle.dump(V,f)

In [162]:
##Calculate entropy
import numpy as np
import os
import scipy.stats as stats
import scipy.io as io
import networkx as nx
import random
import itertools
import glob
from scipy.stats import linregress
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,roc_curve,roc_auc_score
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import seaborn as sns
from scipy.io import loadmat, savemat
#from fracModel import fracOrdUU
from collections import Counter
from collections import defaultdict
from numpy import arange,array,ones,linalg
from sklearn.linear_model import LinearRegression
import pandas as pd
#import powerlaw
import os.path
from os import path
import pickle
import warnings
warnings.filterwarnings("ignore")
import ast

In [163]:
def mm_stats(s,b,k,beta,gamma):
    F_list = []
    en_list=[]
    t_names_list = []
    for i in range(len(s)):
        F_1,en, t_names = Entropy_MC(s[i],b,k,beta,gamma) #Transition Probability vector
        F = F_1 
        F_list.append(F)
        t_names_list.append(t_names)
        #en_list.append(en)
    return F_list,[], t_names_list

In [164]:
def Entropy_MC(s,b,k,beta,gamma):
    #s = s
    inv, l, n = k-b, len(s), 4**k
    T = np.zeros((n,n))
    T_names = np.chararray((n, n), itemsize=10) #np.zeros((n,n))
    count = [0]*n
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd = dict(zip(word_list, list(range(len(word_list)))))
    uu = []
    for i in range(k,l-b):
        n1, n2 = wd[s[i-k:i]], wd[s[i-k+b:i+b]]
        T[n1,n2] += 1
        count[n1] += 1
        T_names[n1,n2] = "{}-{}".format(s[i-k:i], s[i-k+b:i+b])
    wo_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    for i in range(n):
        wo = word_list[i]
        for l in wo_next:
            j = wd[wo[b:]+l]
            T[i,j] = (T[i,j]+beta)/(count[i]+4**b*beta)
            new = "{}-{}".format(wo, wo[b:]+l)
            T_names[i, j] = new
    sum_count = np.sum(count)+4**k*gamma
    prob = [(count[i]+gamma)/sum_count for i in range(len(count))]
    n = len(T)
    H = 0
    F = []
    for i in range(n):
        wo = word_list[i]
        for l in wo_next:
            j = wd[wo[b:]+l]
            F.append(T[i,j])
            H-=prob[i]*T[i,j]*np.log2(T[i,j])
    return F,H, T_names

In [178]:
def ML(b,k,V,beta,gamma,classes):
    k = 4
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    
    X = []
    y = []
    ct = 0
    n = len(V)
    G = [[] for i in range(n)]
    G_names = [[] for i in range(n)]
    X_names = []
    y_names = []
    
    for seq in V:
        for s in seq:
            F_list,en_list, t_names_list = mm_stats(s,b,k,beta,gamma)
            G[ct]+=F_list
            G_names[ct] += t_names_list
        print(len(G[ct]))
        ct+=1
    min_val = float('inf')
    for i in range(n):
        if min_val > len(G[i]):
            min_val = len(G[i])
    
    for i in range(n):
        random.shuffle(G[i])
        X+=[G[i][j] for j in range(len(G[i]))]
        y+=[i for j in range(len(G[i]))]
        X_names = [G_names[i][j] for j in range(len(G_names[i]))]
        y_names = [i for j in range(len(G_names[i]))]
    
    print('Length of X: {}'.format(len(X)))
    X_original = [x_i for x_i in X]
    X = np.array(X)
    y = np.array(y)
    X_original = np.array(X_original)
    X = preprocessing.scale(X)
    U_X = list(X[:min_val])
    U_y = list(y[:min_val])
    T_X = list(X[min_val:len(G[0])])
    T_y = list(y[min_val:len(G[0])])
    
    print(X.shape)
    
    print(y.shape)
    
    print(y)
    
    print(min_val)
    
    print(U_X)
    
    print(U_y)
    
    running_sum = len(G[0])
    for i in range(1,n):
        U_X+=list(X[running_sum:running_sum+min_val])
        U_y+=list(y[running_sum:running_sum+min_val])
        T_X+=list(X[running_sum+min_val:running_sum+len(G[i])])
        T_y+=list(y[running_sum+min_val:running_sum+len(G[i])])
        running_sum+=len(G[i])
    zipped_U = list(zip(U_X,U_y))
    random.shuffle(zipped_U)
    U_X = [x_i for x_i,_ in zipped_U]
    U_y = [y_i for _,y_i in zipped_U]
    zipped_T = list(zip(T_X,T_y))
    random.shuffle(zipped_T)
    T_X = [x_i for x_i,_ in zipped_T]
    T_y = [y_i for _,y_i in zipped_T]
    U_X = np.array(U_X)
    U_y = np.array(U_y)
    T_X = np.array(T_X)
    T_y = np.array(T_y)
    
    print(U_X)
    print(U_y)
    
    print(T_X)
    print(T_y)
    
    return U_X,U_y,X_original,T_X,T_y


In [179]:
import sys
with open('Clade_Analysis/V_clades.pkl','rb') as f:  
    V = pickle.load(f)
l = [[1,4]]
beta = 0.5
gamma = 0.5
R = len(V)
clades = ['20C', '19A'] #['20A','19A','20B','19B','20C']
pairs = [[0,1]] #[[0,1],[0,2],[0,3],[0,4],[1,2],[1,3],[1,4],[2,3],[2,4],[3,4]]
folder = 'Clade_Analysis'
cutoff = -10
#plt.rcParams.update({'font.size': 14})
acc_mean = [[] for i in range(5)]
for val in l:
    b,k = val[0],val[1]
    for p_1,p_2 in pairs:
        f = open(folder+'/'+clades[p_1]+'_vs_'+clades[p_2]+'.txt','w')
        V_1 = [V[p_1],V[p_2]]
        classes = [clades[p_1],clades[p_2]]
        X,y,X_original,T,y_T = ML(b,k,V_1,beta,gamma,classes)
        #mean,std = build_model(X,y,X_original,b,k,classes,cutoff,T,y_T,f)
        #print(clades[p_1]+' vs '+clades[p_2]+':'+str(mean)+'+/-'+str(std))
        #f.write(clades[p_1]+' vs '+clades[p_2]+':'+str(mean)+'+/-'+str(std))
        #f.close()

399
188
Length of X: 587
(587, 1024)
(587,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1